In [ ]:
import os, sys, subprocess, gc, traceback, torch
from pathlib import Path
import numpy as np
import soundfile as sf
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}", flush=True)

# 1. Setup - Targeted FAST installation
log("Checking environment...")
try:
    # Check for espeak-ng
    if subprocess.run("espeak-ng --version", shell=True, capture_output=True).returncode != 0:
        log("Installing espeak-ng...")
        subprocess.run("apt-get update -q; apt-get install -y -q espeak-ng", shell=True)
    
    # Fast minimal installs
    log("Installing chatterbox missing deps...")
    # Skip huge ones (torch, transformers are already on Kaggle)
    # phonemizer, num2words are small
    subprocess.run("pip install -q chatterbox-tts phonemizer num2words --no-deps", shell=True)

    from chatterbox import Chatterbox
    log("Setup complete (Imports done).")
except Exception as e:
    log(f"Setup warning: {e}. Trying full install...")
    subprocess.run("pip install -q chatterbox-tts phonemizer num2words --no-cache-dir", shell=True)
    from chatterbox import Chatterbox

# 2. Parameters (Injected by JobManager)
text = "Final chatterbox test."
voice = "af_sky"
output_file = "chatterbox.wav"

# 3. Model Loading & Verification
log(f"Loading Chatterbox model. Text: {text[:50]}...")
try:
    # Model is Kokoro-82M (very fast, <100MB)
    model = Chatterbox()
    
    log("Generating audio...")
    audio, sr = model.generate(
        text=text,
        voice=voice,
        speed=1.0,
        lang_code="a" # American English
    )
    
    if audio is not None:
        if isinstance(audio, torch.Tensor):
            audio = audio.detach().cpu().float().numpy()
        
        if hasattr(audio, 'ndim') and audio.ndim > 1:
            audio = audio.squeeze()
        
        sf.write(output_file, audio, int(sr))
        log(f"SUCCESS: Audio saved to {output_file}")
    else:
        log("ERROR: No audio data")
        sys.exit(1)

except Exception as e:
    log(f"CRITICAL ERROR: {e}")
    traceback.print_exc()
    sys.exit(1)
finally:
    if 'model' in locals(): del model
    torch.cuda.empty_cache()
    gc.collect()
    log("Job finished.")